In [ ]:
import numpy as np
import pandas as pd
import sys
import tensorflow as tf

## drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 경로 지정

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/kaggle/animation-recommendation_system/data
sys.path.append('/content/drive/MyDrive/Colab Notebooks/kaggle/animation-recommendation_system/data')
print(sys.path)
print(tf.config.list_physical_devices('GPU'))

/content/drive/MyDrive/Colab Notebooks/kaggle/animation-recommendation_system/data
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab Notebooks/kaggle/animation-recommendation_system/data']
[]


# rating데이터 가져오기

In [ ]:
# Importing anime details dataframe
rating_df=pd.read_csv('./rating_complete.csv')
print("Shape of the Dataset:",rating_df.shape)
rating_df.head(3)

Shape of the Dataset: (57633278, 3)


,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7


In [ ]:
# Encoding categorical data
user_ids = rating_df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_df["user"] = rating_df["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_df["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_df["anime"] = rating_df["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min rating: {}, Max rating: {}".format(min(rating_df['rating']), max(rating_df['rating'])))

Num of users: 310059, Num of animes: 16872
Min rating: 1, Max rating: 10


* user와 애니메이션 별로 순서대로 0부터 ~~까지 대응대도록 인코딩.
* 이게 모델에 들어갈 user와 anime의 인덱스가됨
* user2user_encoded와 user_encoded2user의 차이는 서로 반대 버전이라는거임 user_userencoded2user는 인코딩된 값에서 원래의 user_id로 인코딩하는 것
* anime는 위와 동일함

In [ ]:
model =tf.keras.models.load_model('./anime_model.h5')

* 모델 불러오기

In [ ]:
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

anime_weights = extract_weights('anime_embedding', model)
user_weights = extract_weights('user_embedding', model)

* gnf와 mlp모델 학습 결과인 user간의 학습 결과 가중치와 anime간의 학습 결과 가중치를 학습한 모델에서의 임베딩 레이어로부터 가져옴
* 값의 범위가 다를 것이기에 -1~1 사이로 정규화함
* 하지만 우리는 아이템 기반 추천시스템이기에 anime_weughts만 있어도 됨

## 메타 데이터 가져오기

In [ ]:
df_anime = pd.read_csv('./anime.csv', low_memory=True)
df_anime = df_anime.replace("Unknown", np.nan)
print("Shape of the Dataset:",df_anime.shape)
df_anime.head(5)

Shape of the Dataset: (17562, 35)


,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",NaN,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


### 이름 맞추고 데이터 수정

In [ ]:
# Fixing Names
def getAnimeName(anime_id):
    try:
        name = df_anime[df_anime.anime_id == anime_id].eng_version.values[0]
        if name is np.nan:
            name = df_anime[df_anime.anime_id == anime_id].Name.values[0]
    except:
        print('error')

    return name

df_anime['anime_id'] = df_anime['MAL_ID']
df_anime["eng_version"] = df_anime['English name']
df_anime['eng_version'] = df_anime.anime_id.apply(lambda x: getAnimeName(x))

df_anime.sort_values(by=['Score'],
               inplace=True,
               ascending=False,
               kind='quicksort',
               na_position='last')

df_anime = df_anime[["anime_id", "eng_version",
         "Score", "Genres", "Episodes",
         "Type", "Premiered", "Members"]]

* getAnimeName함수는 입력할 rating의 anime_id와 df_anime anime_id가 같으면 df_anime의 eng_name값을 반환하여 rating데이터에 없는 영문명을 반환하도록 하는 함수

* 그 이후는 df_anime의 변수명을 바꿔주고 df_anime의 score를 기준으로 정렬함
* 원하는 컬럼만 지정해서 새롭게 데이터프레임 저장

In [ ]:
def getAnimeFrame(anime):
    if isinstance(anime, int):
        return df_anime[df_anime.anime_id == anime]
    if isinstance(anime, str):
        return df_anime[df_anime.eng_version == anime]

### 시놉시스 데이터 가져오기

In [ ]:
cols = ["MAL_ID", "Name", "Genres", "sypnopsis"]
sypnopsis_df = pd.read_csv('./anime_with_synopsis.csv', usecols=cols)

def getSypnopsis(anime):
    if isinstance(anime, int):
        return sypnopsis_df[sypnopsis_df.MAL_ID == anime].sypnopsis.values[0]
    if isinstance(anime, str):
        return sypnopsis_df[sypnopsis_df.Name == anime].sypnopsis.values[0]

* 시놉시스 데이터 가져오고
* getSypnopsis함수는 입력할 rating의 anime_id와 sypnopsis_df의 MAL_id가 같으면 sypnopsis_df의 sypnopsis값을 반환하여 rating데이터에 없는 시놉시스(애니 요약된 설명)을 반환하도록 하는 함수

## Item 기반 추천

### 영화 3개받고 5개 추천

In [ ]:
def find_similar_animes_combined(anime_names, n=3, return_dist=False, neg=False):

  # try except문을 통해 try문속 오류, false가 뜨면 except문으로 가서 예외를 처리해줌
    try:

        # getAnimeFrame 함수를 사용하여 각 애니메이션 제목에 대응하는 anime_id를 얻고, 이를 anime2anime_encoded 딕셔너리를 통해 인코딩된 인덱스로 변환하여 저장
        # 만들어놓은 리스트로 3개의 입력값들 저장
        encoded_indices = []
        input_anime_ids = []
        for name in anime_names:
            index = getAnimeFrame(name).anime_id.values[0]
            input_anime_ids.append(index)
            encoded_index = anime2anime_encoded.get(index)
            encoded_indices.append(encoded_index)


        #3개의 애니의 임베딩 벡터를 평균하여 하나의 결합 벡터를 생성하고, 이를 정규화함(여기서 임베딩 벡터는 기존의 애니 평점을 학습을 통해 애니메이션 별로 생성된 가중치 벡터를 뜻함)
        combined_weights = np.mean(anime_weights[encoded_indices], axis=0)
        combined_weights = combined_weights / np.linalg.norm(combined_weights)


        #내적 함수를 통해 정규화한 임베딩 벡터와 전체 애니메이션 임베딩 벡터의 유사도를 구함
        dists = np.dot(anime_weights, combined_weights)

        # 유사도 행렬을 정렬함
        sorted_dists = np.argsort(dists)

         # 입력된 애니메이션의 수를 포함하여 n개의 가장 유사하거나 (neg=False) 가장 먼 (neg=True) 애니메이션을 선택함(+를 하는 이유는 기존의 3개가 제일 위에 뜰 것이기에 그걸 고려하고 3개 더 애니를 가져오기 위함임)
        n = n + len(input_anime_ids)

        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]

        print('animes closest to {}'.format(anime_names))

        # 입력 파라미터 중 하나인 return_dist는 True로 지정하면 입력 임베딩 벡터의 유사도 행렬과 입력한 애니를 포함한 6개의 애니메이션을 반환함
        if return_dist:
            return dists, closest


        # df_anime 가져옴
        rindex = df_anime

        # 유사한 3개의 애니메이션 최종 결과 배열
        SimilarityArr = []

        #가장 유사한 3개의 애니를 for문 돌림
        for close in closest:

            # closet는 가장 유사한 애니들이 인코딩 된 anime_id로 가져왔기에 추천 시스템 출력 결과로 영문명과 장르를 추출하려면 기존의 anime_id로 디코딩이 필요함
            decoded_id = anime_encoded2anime.get(close)

            # 추천할 애니 3개의 디코딩한 id가 입력한 애니 3개 id랑 같으면 continue를 통해 for문을 스킵하게하여 출력 결과에 입력한 애니3개를 나오지 않도록 함
            if decoded_id in input_anime_ids:
                continue


            #getSypnopsis함수로 추천할 애니 3개의 시놉시스 가져옴
            sypnopsis = getSypnopsis(decoded_id)

            #getAnimeFrame함수로 추천할 애니 id에 해당하는 df_anime의 데이터프레임 값을 가져옴
            anime_frame = getAnimeFrame(decoded_id)

            # 가져온 데이터프레임의 이름과 장르를 추출
            anime_name = anime_frame.eng_version.values[0]
            genre = anime_frame.Genres.values[0]

            #유사도는 구한 유사도 행렬에서 인코딩 anime_id로 가져옴
            similarity = dists[close]

            #추천할 애니메이션 배열에 추가
            SimilarityArr.append({"anime_id": decoded_id, "name": anime_name,
                                  "similarity": similarity, "genre": genre,
                                  'sypnopsis': sypnopsis})

        #최종 추천 애니메이션 배열 dataframe으로 바꾸고 유사도를 기반으로 정렬
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", ascending=False)

        #anime_id는 없어도 되니 빼고 지정한 n개의 값 출력하도록 반환
        return Frame.drop(['anime_id'], axis=1).head(n - len(input_anime_ids))  # Return top 'n' results excluding input animes

    #코드 오류 및 데이터에 입력하 애니 없을 떄 예외 처리
    except Exception as e:
        print('{}!, Not Found in Anime list'.format(anime_names))
        print(str(e))
        return pd.DataFrame()


In [ ]:
# Example usage
anime_list = ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']
find_similar_animes_combined(anime_list, n=3)

animes closest to ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']


,name,similarity,genre,sypnopsis
3,Your Name.,0.701476,"Romance, Supernatural, School, Drama","suha Miyamizu, a high school girl, yearns to l..."
2,Your Lie in April,0.695971,"Drama, Music, Romance, School, Shounen",usic accompanies the path of the human metrono...
1,ERASED,0.687197,"Mystery, Psychological, Supernatural, Seinen","hen tragedy is about to strike, Satoru Fujinum..."
